In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size =4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

cuda


In [ ]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))

vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [ ]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)
x, y = get_batch('train')
print('inputs:')
print(x.shape)
print(x)
print('targets:')
print(y)


inputs:
torch.Size([4, 8])
tensor([[58,  1, 64, 62, 73, 73, 58, 67],
        [ 1,  1,  1,  1,  1,  1,  1, 40],
        [ 0,  0,  3, 47, 58,  1, 57, 62],
        [68, 76,  9,  1, 76, 61, 58, 73]], device='cuda:0')
targets:
tensor([[ 1, 64, 62, 73, 73, 58, 67,  1],
        [ 1,  1,  1,  1,  1,  1, 40, 25],
        [ 0,  3, 47, 58,  1, 57, 62, 57],
        [76,  9,  1, 76, 61, 58, 73, 61]], device='cuda:0')


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

    
    


3c5VCSn8jmCNFlx﻿'2A.rYO,x9ri?YO91A4pH(lI﻿Sj;T9(ul;p3[eB_*?lmaEI"WXLQ?[d1!c.B5jYj)y_ rM_NXh5QLg_iMHtSV8g)g2ltLZN3dP5t?_S_,'GVCoWKMqti8y5d",yZ&Erlmg5v(YG2F5aY:V1)QKpYPYs*y5na'ofc5a0x&E﻿vwl'KMS- AAAsjkI﻿Igw;AuDJ;*A]b,sI:NFdMfJ:aKls]u:Pj.IPerM!fj4M[ztSxAu[a'HOPvuZ73Zb
"﻿:
y:DPE[z&L zL8pwh!cq9qrcE]p"XT
SZUZJeqtT(CZ6iH2liyCh&I﻿""Z
42lJ:8pUT﻿]L8gH_MM?SnQ7XW"GGb
jH)0!?R7-i9y﻿'5a-
u:zjsGFRDE46,K2bG8Eq9DF3GVd1XR;Z8y8oH_:8WO-g25.x7] HS7gAq)
]VOEx]GVpg3;V(SQ?;a
LkzKM19X*D[edxHB!_Mjs-:QLU6]zerKu:phnU 1erTl'!


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


step: 0, train loss: 4.4534, val loss: 4.4531
step: 250, train loss: 4.3750, val loss: 4.3873
step: 500, train loss: 4.3435, val loss: 4.3471
step: 750, train loss: 4.2929, val loss: 4.2956
4.336724281311035
